In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import numpy as np

# Load the dataset
file_path = 'Crop_Dataset.csv'
crop_data = pd.read_csv(file_path)

# Features and target
X = crop_data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = crop_data['Label_Encoded']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model
model_filename = 'crop_recommendation_model_new.pkl'
joblib.dump(model, model_filename)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

accuracy, model_filename


(0.9931818181818182, 'crop_recommendation_model_new.pkl')

In [6]:
import requests

# Define the URL for the API
url = 'http://127.0.0.1:5000/predict'

# Define the input data
data = {
    'N': 20,
    'P': 42,
    'K': 43,
    'temperature': 90.88,
    'humidity': 82.00,
    'ph': 6.50,
    'rainfall': 62.94
}

# Send a POST request
response = requests.post(url, json=data)

# Print the response
if response.status_code == 200:
    print('Top 3 recommended crops:', response.json()['top_3_crops'])
else:
    print('Error:', response.status_code, response.text)


Top 3 recommended crops: ['oats', 'sorghum', 'peas']


In [1]:
import base64
import requests
import json

def get_base64_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def send_request_to_openai(query, image_path, api_key):
    # Convert the image to Base64
    base64_image = get_base64_image(image_path)
    
    # API endpoint
    url = "https://api.openai.com/v1/chat/completions"
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    # Data payload
    data = {
        "model": "gpt-4o-2024-05-13",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    },
                    {
                        "type": "text",
                        "text": query
                    }
                ]
            }
        ],
        "temperature": 1,
        "max_tokens": 256,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
    
    # Send the request
    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    return response.json()

# Example usage
api_key = "API_KEY"
query = "Can you describe this image?"
image_path = "1.jpg"

response = send_request_to_openai(query, image_path, api_key)
print(response)


{'id': 'chatcmpl-9YeCkVTmRpsm4qV5q658mStQGWfJ6', 'object': 'chat.completion', 'created': 1718044726, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'This image features a bottle of a perfume labeled "Club De Nuit Precieux I". The bottle is sleek and predominantly black with silver accents, including a chain with a small rectangle tag hanging from the neck, also inscribed with "Club De Nuit". To the right of the bottle, there are images of bergamot (a green citrus fruit sliced in half), jasmine flowers (white flowers with yellow details inside), and amber (a translucent, golden-brown gemstone-like substance). The composition implies that these are key notes or ingredients in the fragrance. The image also has the website "ifraganceofficial.com" printed on the bottom left corner of the bottle.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 778, 'completion_tokens': 132, 'total_tokens': 910}, 'system_fingerprin